## Imports


In [1]:
import pandas as pd
import numpy as np
import requests
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from scipy.stats import poisson
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from datetime import datetime
from datetime import date
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import sasoptpy as so
import os
from IPython.display import clear_output

pd.set_option('display.max_columns', 300)

## Fixture Generating Funtions
Generate ticker for visualization and optimization, based on the live fixture info from the Premier League API and any custom fixture timings and probabilities set by the user

In [2]:
## Generate matchday ticker dataframe, team_fixtures
def generate_ticker(gw_range=None, exclude_teams=None, custom_fixtures=None, extra_fixtures=None, generate_all_dataframes=False):

    # Infer fixture difficulties
    team_priors = pd.read_csv(f'../data/team_priors.csv')
    team_priors['h_off'] = round(team_priors['bl_g_for'] * team_priors['home_adv_g'],2)
    team_priors['h_def'] = round(team_priors['bl_g_against'] / team_priors['home_adv_g'],2)
    team_priors['h_gd'] = team_priors['h_off'] - team_priors['h_def']
    team_priors['a_off'] = round(team_priors['bl_g_for'] / team_priors['home_adv_g'],2)
    team_priors['a_def'] = round(team_priors['bl_g_against'] * team_priors['home_adv_g'],2)
    team_priors['a_gd'] = team_priors['a_off'] - team_priors['a_def']

    # Get fixtures and team data from pl api
    r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
    fpl_data = r.json()
    team_data = pd.DataFrame(fpl_data['teams'])
    team_data = team_data[['id', 'short_name']].rename(columns={"id": "team_id"})
    team_data = team_data.replace('NFO', 'FOR')
    r = requests.get('https://fantasy.premierleague.com/api/fixtures/')
    fixtures_data = r.json()
    fixtures_data = pd.DataFrame(fixtures_data)
    fixtures_data = fixtures_data.drop('stats', axis=1)
    fixtures_data = fixtures_data[fixtures_data['started'] != True]
    # fixtures_data.to_csv('../data/fixtures_test_all.csv')
    fixtures_data = fixtures_data[fixtures_data['started'] == False]
    fixtures_data['gw'] = fixtures_data['event'].astype(int)
    fixtures_data['kickoff_time'] = pd.to_datetime(fixtures_data['kickoff_time'])
    fixtures_data['datetime'] = fixtures_data['kickoff_time'].dt.tz_convert('UTC').dt.tz_localize(None)
    fixtures_data['date_str'] = fixtures_data['datetime'].dt.strftime('%Y-%m-%d %H:%M')
    fixtures_data['time_str'] = fixtures_data['datetime'].dt.strftime('%H:%M')
    fixtures_data = pd.merge(fixtures_data, team_data, left_on='team_a', right_on='team_id', how='left').rename(columns={"short_name": "team_a_name", "team_id": "team_a_id"})
    fixtures_data = pd.merge(fixtures_data, team_data, left_on='team_h', right_on='team_id', how='left').rename(columns={"short_name": "team_h_name", "team_id": "team_h_id"})

    # Add customized fixtures to fixtures table
    fixtures_data.loc[:,'customized'] = False
    fixtures_data['custom_dates'] = [[] for _ in range(len(fixtures_data))]
    fixtures_data['custom_probs'] = [[] for _ in range(len(fixtures_data))]
    if custom_fixtures is not None:
        custom_fixtures['added_to_ticker'] = False
        for index, row in custom_fixtures.iterrows():
            h = custom_fixtures.loc[index,'home_team']
            a = custom_fixtures.loc[index,'away_team']
            listy = fixtures_data.index[(fixtures_data['team_h_name'] == h) & (fixtures_data['team_a_name'] == a)].to_list()
            # if the fixture to be added isn't in the fixtures to be played, add it
            if listy != []:
                i = listy[0]
                fixtures_data.at[i, 'customized'] = True
                fixtures_data.at[i, 'custom_dates'] = custom_fixtures.loc[index,'dates']
                fixtures_data.at[i, 'custom_probs'] = custom_fixtures.loc[index,'probabilities']
                custom_fixtures.at[index, 'added_to_ticker'] = True

    # Drop rows not in gameweek range
    if gw_range is not None:
        mask = fixtures_data['gw'].isin(gw_range)
        fixtures_data = fixtures_data[mask]

    # Generate ticker
    natural_fix_dates = sorted(fixtures_data['date_str'].unique())
    custom_fix_dates = []
    if custom_fixtures is not None:
        for i, x in custom_fixtures.iterrows():
            custom_fix_dates += (custom_fixtures.loc[i, 'dates'])
    unique_dates = sorted(natural_fix_dates + custom_fix_dates)
    unique_dates = sorted(list(set(unique_dates)))
    team_fixtures = team_data.assign(**dict.fromkeys(unique_dates, ''))
    old_date = None
    old_datetime = None
    for index, row in fixtures_data.iterrows():
        new_date = row['date_str']
        new_datetime = row['datetime']
        away_team = row['team_a_name']
        home_team = row['team_h_name'].lower()
        if old_date != new_date or (row['datetime'] == old_datetime and first_fix):
            away_team += '!'
            home_team += '!'
            first_fix = True
        else:
            first_fix = False
        team_fixtures.loc[team_fixtures['short_name'] == row['team_h_name'], row['date_str']] = away_team
        team_fixtures.loc[team_fixtures['short_name'] == row['team_a_name'], row['date_str']] = home_team
        old_date = new_date
        old_datetime = new_datetime
    copied_natural_fixtures = team_fixtures.copy()
    # Add the custom fixtures to the ticker, deleting their 'natural' placement
    # NB: only fixtures that have yet to be played can be added
    if custom_fixtures is not None:
        for index, row in fixtures_data.iterrows():
            if row['customized']:
                team_fixtures.loc[team_fixtures['short_name'] == row['team_h_name'], row['date_str']] = ''
                team_fixtures.loc[team_fixtures['short_name'] == row['team_a_name'], row['date_str']] = ''
                for i, x in enumerate(row['custom_probs']):
                    if x == 1:
                        prob_str = ''
                    elif x == 0:
                        break
                    else:
                        prob_str = '*' + str(int(x*100)) + '%' 
                    date = row['custom_dates'][i]      
                    away_team = row['team_a_name'] + '!' + prob_str
                    home_team = row['team_h_name'].lower() + '!' + prob_str
                    if team_fixtures.loc[team_fixtures['short_name'] == row['team_h_name'], date].to_list()[0] != '':
                        away_team = '\n' + away_team
                    if team_fixtures.loc[team_fixtures['short_name'] == row['team_a_name'], date].to_list()[0] != '':
                        home_team = '\n' + home_team
                    team_fixtures.loc[team_fixtures['short_name'] == row['team_h_name'], date] += away_team
                    team_fixtures.loc[team_fixtures['short_name'] == row['team_a_name'], date] += home_team
        # Add those fixtures which aren't included in the natural fixtures
        if False in custom_fixtures['added_to_ticker'].tolist():
            extra_custom_fixtures = custom_fixtures.loc[custom_fixtures['added_to_ticker'] == False]
            for index, row in extra_custom_fixtures.iterrows():
                for i, x in enumerate(row['probabilities']):
                    if x == 1:
                        prob_str = ''
                    else:
                        prob_str = '*' + str(int(x*100)) + '%' 
                    date = row['dates'][i]      
                    away_team = row['away_team'] + '!' + prob_str
                    home_team = row['home_team'].lower() + '!' + prob_str
                    if team_fixtures.loc[team_fixtures['short_name'] == row.loc['home_team'], date].to_list() != []:
                        if team_fixtures.loc[team_fixtures['short_name'] == row.loc['home_team'], date].to_list()[0] != '':
                            away_team = '\n' + away_team
                    if team_fixtures.loc[team_fixtures['short_name'] == row.loc['away_team'], date].to_list() != []:
                        if team_fixtures.loc[team_fixtures['short_name'] == row.loc['away_team'], date].to_list()[0] != '':
                            home_team = '\n' + home_team
                    prev1 = team_fixtures.loc[team_fixtures['short_name'] == row.loc['home_team'], date] + away_team
                    prev2 = team_fixtures.loc[team_fixtures['short_name'] == row.loc['away_team'], date] + home_team
                    team_fixtures.loc[team_fixtures['short_name'] == row.loc['home_team'], date] = prev1
                    team_fixtures.loc[team_fixtures['short_name'] == row.loc['away_team'], date] = prev2
                    custom_fixtures.loc[index, 'added_to_ticker'] = True

    # Generate matchday mapping to tff gw, fpl gw, date, and day of week
    weekdays = []
    matchdays = []
    tff_gw_list = []
    fpl_gw_list = []
    tff_gw_df = pd.read_csv('../data/tff_gw_starts_2324.csv')
    fpl_gw_df = pd.read_csv('../data/fpl_gw_starts_2324.csv')
    tff_gw_date = tff_gw_df.loc[0,'start_date']
    fpl_gw_date = fpl_gw_df.loc[0,'start_date']
    fpl_gw_index = 0
    tff_gw_index = 0
    # loop through matchdays
    for i, x in enumerate(unique_dates):
        new_day = str(datetime.strptime(unique_dates[i], '%Y-%m-%d %H:%M').date().weekday())
        weekdays.append(new_day)
        matchdays.append(i+1)
        # while the date of the current matchday is later than that of the proposed tff gw, proceed to the date of the next tff gw
        while x >= tff_gw_date:
            broken = False
            if tff_gw_index > len(tff_gw_df)-1:
                broken = True
                break
            tff_gw_index += 1
            tff_gw_date = tff_gw_df.loc[tff_gw_index-1, 'start_date']
        if broken:
            tff_gw = tff_gw + 1
        else:
            tff_gw = tff_gw_df.loc[tff_gw_index-1, 'gameweek']-1
        tff_gw_list.append(tff_gw)
        # while the date of the current matchday is later than that of the proposed fpl gw, proceed to the date of the next fpl gw
        while x >= fpl_gw_date:
            broken = False
            if fpl_gw_index > len(fpl_gw_df)-1:
                broken = True
                break
            fpl_gw_index += 1
            fpl_gw_date = fpl_gw_df.loc[fpl_gw_index-1, 'start_date']
        if broken:
            fpl_gw = fpl_gw + 1
        else:
            fpl_gw = fpl_gw_df.loc[fpl_gw_index-1, 'gameweek']-1
        fpl_gw_list.append(fpl_gw)
    data = {'unique_dates': unique_dates,
            'weekday': weekdays,
            'matchday': matchdays,
            'tff_gw': tff_gw_list,
            'fpl_gw': fpl_gw_list
            }
    md_map = pd.DataFrame(data)
    date_0 = md_map.loc[0,'unique_dates']
    date_0 = datetime.strptime(date_0, '%Y-%m-%d %H:%M').date()
    for i, col in md_map.iterrows():
        date_1 = md_map.loc[i,'unique_dates']
        date_1 = datetime.strptime(date_1, '%Y-%m-%d %H:%M').date()
        delta = int((date_1 - date_0).days)
        md_map.loc[i,'days_elapsed'] = delta

    # Generate dataframes for all possible fixture permutations for stochastic optimization, assuming all fixtures are independent
    if generate_all_dataframes and extra_fixtures is not None:
        uncertain_fixtures = extra_fixtures.drop(extra_fixtures[extra_fixtures.probability == 1].index)
        my_list = uncertain_fixtures.probability.tolist()
        # Assume all fixtures are independent
        n_uncert_fix = len(my_list)
        number_of_permutations = 2**(n_uncert_fix)
        fix_permutation_dict = {}
        for i in range(number_of_permutations):
            fixture_key = f'permutation_{i+1}'
            permutation_string = format(i, f'0{n_uncert_fix}b')
            df = copied_natural_fixtures.copy()
            likelihood = 1
            for i, x in enumerate(permutation_string):
                if bool(int(x)):
                    away_team = uncertain_fixtures.loc[i,'away_team']
                    home_team = uncertain_fixtures.loc[i,'home_team']
                    df.loc[df['short_name'] == home_team, uncertain_fixtures.loc[i,'date']] = away_team
                    df.loc[df['short_name'] == away_team, uncertain_fixtures.loc[i,'date']] = home_team.lower()
                    likelihood = likelihood * uncertain_fixtures.loc[i,'probability']
                else:
                    likelihood = likelihood * (1-uncertain_fixtures.loc[i,'probability'])
            fix_permutation_dict[fixture_key] = {'df': df, 'likelihood': likelihood}
    else:
        fix_permutation_dict = None

    fixtures_df = team_fixtures.copy()

    # Drop excluded teams
    if exclude_teams is not None:
        for i in exclude_teams:
            team_fixtures = team_fixtures[team_fixtures.short_name != i]

    # Add gameweek superheader
    date_to_gw = fixtures_data[['date_str','gw']].drop_duplicates()
    headers = list(team_fixtures.columns.values)
    tff_gw_header = []
    fpl_gw_header = []
    for i in headers:
        if i.startswith('20') == False:
            j = 'tff_gw'
            k = 'fpl_gw'
        else:
            tff_gw = md_map.loc[md_map['unique_dates']==i,'tff_gw'].values[0]
            fpl_gw = md_map.loc[md_map['unique_dates']==i,'fpl_gw'].values[0]
            j = str(tff_gw)
            k = str(fpl_gw)
        tff_gw_header.append(j)
        fpl_gw_header.append(k)
    md_header = []
    for i in headers:
        if i.startswith('20') == False:
            j = 'matchday'
        else:
            md = md_map.loc[md_map['unique_dates']==i,'matchday'].values[0]
            j = str(md)
        md_header.append(j)
    team_fixtures.columns=[tff_gw_header, fpl_gw_header, md_header, headers]


    formatted_fixtures = team_fixtures.copy()
    # Make color map dictionary and function
    color_ts = team_priors[['short_team','h_gd', 'a_gd']].copy()
    min_gd = min(color_ts['h_gd'].values.tolist() + color_ts['a_gd'].values.tolist())*2.3
    max_gd = max(color_ts['h_gd'].values.tolist() + color_ts['a_gd'].values.tolist())#*1.8
    norm = matplotlib.colors.Normalize(vmin=min_gd, vmax=max_gd, clip=True)
    mapper = plt.cm.ScalarMappable(norm=norm, cmap=plt.cm.viridis_r)
    color_ts['h_gd_color'] = color_ts['h_gd'].apply(lambda x: mcolors.to_hex(mapper.to_rgba(x)))
    color_ts['a_gd_color'] = color_ts['a_gd'].apply(lambda x: mcolors.to_hex(mapper.to_rgba(x)))
    h_teams = color_ts['short_team'].values.tolist()
    a_teams = [team.lower() for team in h_teams]
    teams = h_teams + a_teams
    team_gd = color_ts['a_gd_color'].values.tolist() + color_ts['h_gd_color'].values.tolist()
    color_dict = {teams[i]: team_gd[i] for i in range(len(teams))}
    def color_col(col, pattern_map, default=''):
        return np.select(
            [col.str.contains(k, na=False) for k in pattern_map.keys()],
            [f'background-color: {v}' for v in pattern_map.values()],
            default=default
        ).astype(str)
    # Apply styles
    formatted_fixtures = formatted_fixtures.style.apply(color_col,
                                                pattern_map=color_dict
                                                , subset=team_fixtures.columns[2:]
                                                )
    formatted_fixtures = formatted_fixtures.set_table_styles([
                        {'selector': 'th.col_heading', 'props': 'text-align: left;'},
                        {'selector': 'th.col_heading.level0', 'props': 'font-size: 1em;'},
                        {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
                    ], overwrite=False)

    return {'formatted_fixtures': formatted_fixtures, 'fixtures_data': fixtures_df, 'matchday_map': md_map, 'fix_permutation_dict': fix_permutation_dict, 'unformatted_fixtures': team_fixtures}

## Modelling Functions

Generate dataframe of expected points values for a specified period, based on prior team and player level data. Calls fixture fixture generator function

In [3]:
def prior_team_data_gen():

    team_data = pd.read_csv('../data/team_priors.csv', index_col=0)

    return team_data

def prior_player_data_gen(team_data):
    
    prior_player_data = pd.read_csv('../data/prior_player_data.csv')
    pen_taker_override = pd.read_csv('../data/pen_taker_override.csv')
    for index, row in pen_taker_override.iterrows():
        if row['pen_share']==row['pen_share']:
            prior_player_data.loc[prior_player_data['sky_id']==row['sky_id'], 'on_pens'] = row['pen_share']
            print(str(row['reference_name']) + ' pen share overridden to ' + str(row['pen_share']))

    try:
        filepath = '../data/fplreview.csv'
        fplreview = pd.read_csv(filepath)
        fplreview = fplreview.rename(columns={'ID': 'fpl_id'})
        review_xmins = True
        print(f"Using minutes from {filepath}")
    except:
        review_xmins = False
        print(f"{filepath} not found, using default baseline minutes") 

    if review_xmins:
        # Get gw x_mins from fplreview file, and overwite
        review_gw_list = []
        for element in list(fplreview.columns.values):
            if '_xMins' in element:
                review_gw_list.append(element)
        for element in list(prior_player_data.columns.values):
            if element in review_gw_list:
                prior_player_data = prior_player_data.drop(columns = [element])
        prior_player_data = pd.merge(prior_player_data, fplreview.loc[:,['fpl_id'] + review_gw_list], on=['fpl_id'], how='inner')
    else:
        for gw in range(1,39):
            prior_player_data[str(gw)+'_xMins'] = prior_player_data['bl_xmin']
    
    return {'prior_player_data':prior_player_data, 'review_xmins':review_xmins}

def tff_xP_calc(sky_id, opp_team, prior_player_data, team_data, xMins, xP_breakdown=False, review_xmins=True):
    player_data = prior_player_data.loc[prior_player_data['sky_id'] == sky_id].reset_index()
    own_team = player_data.loc[0, 'short_team']
    own_team_data = team_data.loc[team_data['short_team'] == own_team].reset_index()
    opp_data = team_data.loc[team_data['short_team'] == opp_team.upper()].reset_index()
    Pos = player_data.loc[0, 'sky_pos']
    if opp_team.isupper() == True:
        home_adv = own_team_data.loc[0, 'home_adv_g']
        home_adv_pass = own_team_data.loc[0, 'home_adv_pass']
    elif opp_team.islower() == True:
        home_adv = 1/own_team_data.loc[0, 'home_adv_g']
        home_adv_pass = 1/own_team_data.loc[0, 'home_adv_pass']
    else:
        home_adv = 1
        home_adv_pass = 1
    if Pos == 'GK':
        k_G = 5
        k_CS = 5
        k_pCS = 2
        k_2GC = -1
    elif Pos == 'DEF':
        k_G = 5
        k_CS = 5
        k_pCS = 2
        k_2GC = -1
    elif Pos == 'MID':
        k_G = 5
        k_CS = 0
        k_pCS = 0
        k_2GC = 0
    else:
        k_G = 5
        k_CS = 0
        k_pCS = 0
        k_2GC = 0
    k_1GC = 0
    k_Start = 2
    k_Sub = 1
    k_A = 3
    k_PenSv = 5
    k_PenMiss = -2
    k_Yc = -1
    k_Rc = -3
    k_OG = -3
    k_Tk = 0.5
    k_Sv = 0.5
    x_90s = xMins/90
    if review_xmins:
        x_90s = xMins/95
    p_start = (0.5)*(0.5 + np.cbrt((x_90s-0.5)/4)) + (0.5)*x_90s
    StartxP = k_Start * p_start
    SubxP = k_Sub * (1-p_start) * x_90s
    GxP = k_G * player_data.loc[0, 'bl_npxg'] * opp_data.loc[0, 'bl_xg_against_k'] * x_90s * home_adv * player_data.loc[0, 'fin_skill']
    AxP = k_A * player_data.loc[0, 'bl_a'] * opp_data.loc[0, 'bl_g_against_k'] * x_90s * home_adv
    OGxP = k_OG * player_data.loc[0, 'bl_og'] * opp_data.loc[0, 'bl_og_against_k'] * x_90s / home_adv
    PenScorexP = k_G * player_data.loc[0, 'on_pens'] * (player_data.loc[0, 'fin_skill'] * 0.78) * own_team_data.loc[0, 'bl_pk_att_for'] * opp_data.loc[0, 'bl_pk_att_against_k'] * x_90s * home_adv
    PenMissxP = k_PenMiss * player_data.loc[0, 'on_pens'] * (1-player_data.loc[0, 'fin_skill'] * 0.78) * own_team_data.loc[0, 'bl_pk_att_for'] * opp_data.loc[0, 'bl_pk_att_against_k'] * x_90s * home_adv
    if Pos == 'GK':
        PenSvxP = k_PenSv * own_team_data.loc[0, 'bl_pk_att_against_k'] * 0.11 * opp_data.loc[0, 'bl_pk_att_for'] * x_90s / home_adv
        SavexP = k_Sv * player_data.loc[0, 'bl_sv_per_sot'] * opp_data.loc[0, 'bl_sot_for'] * own_team_data.loc[0, 'bl_sot_against_k']  * x_90s / home_adv
        TackxP = 0
    if Pos == 'MID':
        TackxP = k_Tk * player_data.loc[0, 'bl_tack'] * opp_data.loc[0, 'bl_tack_against_k'] * x_90s
        PenSvxP = 0 
        SavexP = 0 
    else:
        PenSvxP = 0    
        TackxP = 0  
        SavexP = 0  
    YcxP = k_Yc * player_data.loc[0, 'bl_yc'] * opp_data.loc[0, 'bl_yc_against_k'] * x_90s
    RcxP = k_Rc * player_data.loc[0, 'bl_rc'] * opp_data.loc[0, 'bl_yc_against_k'] * x_90s
    mu_gc = own_team_data.loc[0, 'bl_g_against_k'] * opp_data.loc[0, 'bl_g_for'] / home_adv
    full_CSxP = k_CS * poisson.cdf(k=0, mu=mu_gc*x_90s) * p_start * player_data.loc[0, 'bl_p_60_given_start'] 
    # below line is the chance of getting below 60 mins given a start, plus the chance of getting a substitute appearance (assumed below 60 mins)
    p_mins_under_60 = p_start * (1-player_data.loc[0, 'bl_p_60_given_start']) + (1-p_start) * (1-player_data.loc[0, 'bl_p_90_given_start'])
    partial_CSxP = k_pCS * poisson.cdf(k=0, mu=mu_gc) * p_mins_under_60
    # this last line assumes you just want all clean sheet points summarized 
    CSxP = full_CSxP + partial_CSxP
    GCxP = k_2GC * ((1 - poisson.cdf(k=1, mu=mu_gc*x_90s)) + (1 - poisson.cdf(k=2, mu=mu_gc*x_90s)) + (1 - poisson.cdf(k=3, mu=mu_gc*x_90s)) + (1 - poisson.cdf(k=4, mu=mu_gc*x_90s)))
    xP = GxP + AxP + OGxP + PenScorexP + PenMissxP + PenSvxP + StartxP + SubxP + YcxP + RcxP + CSxP + GCxP + SavexP + TackxP
    if xP_breakdown == True:
        xP_breakdown = {'Actions': ['Start', 'Sub', 'Goal', 'Assist', 'Own Goal', 'Pen Goal', 'Pen Miss', 'Pen Save', 'Yellow Card', 'Red Card', 'Clean Sheet', 'Goal Conceded',
                                    'Save', 'Tackle', 'TOTAL'],
                        'xP': [StartxP, SubxP, GxP, AxP, OGxP, PenScorexP, PenMissxP, PenSvxP, YcxP, RcxP, CSxP, GCxP, SavexP, TackxP, xP]
                        }
        xP_breakdown = pd.DataFrame(xP_breakdown)
        xP_breakdown = xP_breakdown.drop(xP_breakdown[xP_breakdown.xP == 0].index)
        xP_breakdown.xP = round(xP_breakdown.xP,2)
    return {'xP': xP, 'xP_breakdown': xP_breakdown}

def generate_model_output(first_md=1, last_md=14, filename_suffix=None, custom_fixtures=None, teamsheet_boost=None):
    schedule_name = 'sky_schedule'
    if filename_suffix is not None:
        schedule_name += filename_suffix
    
    if custom_fixtures is not None:
        r = generate_ticker(custom_fixtures=custom_fixtures)
    else:
        r = generate_ticker()
    md_map_2 = r['matchday_map']
    sky_schedule_2 = r['fixtures_data']
    formatted_fixtures = r['formatted_fixtures']
    unformatted_fixtures = r['unformatted_fixtures']
    headers = []
    for i, x in enumerate(sky_schedule_2.columns.values.tolist()):
        if x in md_map_2['unique_dates'].tolist():
            h = md_map_2.loc[md_map_2['unique_dates'] == x, 'matchday'].values[0]
            h = 'MD ' + str(h)
            headers.append(h)
        else:
            headers.append(x)
    sky_schedule_2.columns = headers

    if str(last_md) == last_md:
        if last_md > md_map_2['unique_dates'].tolist()[-1]:
            last_md = md_map_2.loc[len(md_map_2)-1, 'matchday']
        else:
            for i, x in enumerate(md_map_2['unique_dates'].tolist()):
                if last_md < x:
                    last_md = md_map_2.loc[i, 'matchday'] - 1
                    break
    if str(last_md) == last_md:
        last_md = md_map_2.loc[len(md_map_2)-1, 'matchday']

    matchdays = range(first_md,last_md+1)
    team_data = prior_team_data_gen()
    player_data_results = prior_player_data_gen(team_data)
    prior_player_data = player_data_results['prior_player_data']
    review_xmins = player_data_results['review_xmins']
    # prior_player_data = player_data['prior_player_data']

    fpd = pd.merge(prior_player_data, sky_schedule_2, left_on='short_team', right_on='short_name', how='left')
    
    fixture_player_data = fpd.copy()
    for i in range(first_md, last_md+1):
        gw = md_map_2.loc[md_map_2['matchday']==i, 'fpl_gw'].values[0]
        if f'{gw}_xMins' not in fixture_player_data.columns:
            fixture_player_data[f'{gw}_xMins'] = fixture_player_data[f'{gw-1}_xMins']
        fixture_player_data[f'MD {i} Game'] = fixture_player_data[f'MD {i}'].str.len() > 1.5
        fixture_player_data[f'MD_{i}_xMins'] = fixture_player_data[f'{gw}_xMins'] * fixture_player_data[f'MD {i} Game']
    players = fixture_player_data.index.tolist()
    for p in players:
        SKY_ID = fixture_player_data.loc[p, 'sky_id']
        for m in matchdays:
            xMins = fixture_player_data.loc[p, f'MD_{m}_xMins']
            if xMins < 5:
                xP = 0
            else:
                fix_string = fixture_player_data.loc[p, f'MD {m}']
                if '\n' in fix_string:
                    xP = 0
                    fix_list = fix_string.split('\n')
                    for i, x in enumerate(fix_list):
                        r = tff_xP_calc(SKY_ID, x[:3], fixture_player_data, team_data, xMins, xP_breakdown=False, review_xmins=review_xmins)
                        sub_xP = r['xP']
                        if any(c.isdigit() for c in x):
                            sub_xP = sub_xP * int(''.join(filter(str.isdigit, x))) / 100
                        xP += sub_xP
                else:
                    r = tff_xP_calc(SKY_ID, fixture_player_data.loc[p, f'MD {m}'][:3], fixture_player_data, team_data, xMins, xP_breakdown=False)
                    xP = r['xP']
                    if any(c.isdigit() for c in fix_string):
                        xP = xP * int(''.join(filter(str.isdigit, fix_string))) / 100
                if teamsheet_boost is not None:
                    if '!' in fix_string:
                        xP = xP * (1+teamsheet_boost)
            fixture_player_data.loc[p, f'MD_{m}_Pts'] = round(xP, 2)
    skymodel_output = pd.concat([fixture_player_data.loc[:,['sky_id', 'fbref_player', 'short_team', 'sky_pos', 'tff_value']],
                                    fixture_player_data.iloc[:,-(last_md-first_md+1):]],axis = 1)
    skymodel_output['Total_Pts'] = skymodel_output.iloc[:, -(last_md-first_md+1):].sum(axis=1)
    skymodel_output = skymodel_output.fillna(0)
    filename = 'skymodel_output'
    if filename_suffix is not None:
        filename += filename_suffix
    skymodel_output.to_csv(f'../data/{filename}.csv')
    md_map_2.to_csv(f'../data/md_map.csv', index = False)

    return {'skymodel_output':skymodel_output, 'formatted_fixtures':formatted_fixtures, 'md_map': md_map_2, 'unformatted_fixtures': unformatted_fixtures}

def read_fpl_kid_model(filepath = '../data/fplkid.csv'):
    fpl_kid = pd.read_csv(filepath).fillna(0)
    fpl_kid = fpl_kid.rename(columns = {'Pos':'sky_pos', 'ID':'sky_id', 'BV':'tff_value'})
    fpl_kid['Total_Pts'] = 0
    for i in range(200):
        fpl_kid = fpl_kid.rename(columns = {f'GD{i}_Pts':f'MD_{i}_Pts',f'GD{i}_xMins':f'MD_{i}_xMins'})
        if f'MD_{i}_Pts' in fpl_kid.columns.tolist():
            fpl_kid['Total_Pts'] += fpl_kid[f'MD_{i}_Pts']
    fpl_kid = fpl_kid.drop(columns=['Name', 'SV', 'Team'])

    fpl_kid['sky_pos'] = fpl_kid['sky_pos'].str.replace('G','GK')
    fpl_kid['sky_pos'] = fpl_kid['sky_pos'].str.replace('F','FOR')
    fpl_kid['sky_pos'] = fpl_kid['sky_pos'].str.replace('D','DEF')
    fpl_kid['sky_pos'] = fpl_kid['sky_pos'].str.replace('M','MID')

    player_data_merge = pd.read_csv('../data/prior_player_data.csv')
    player_data_merge = player_data_merge[['sky_id','fbref_player','short_team']]

    fpl_kid = pd.merge(left = fpl_kid, right = player_data_merge, on='sky_id', how='inner')
    fpl_kid = fpl_kid.set_index('sky_id')
    return fpl_kid


## Optimization Functions

Generate optimal solution for team or analyze multiple simulated runs with noise, based on the model output

In [4]:
data = {'sky_pos': ['GK', 'DEF', 'MID', 'FOR'],
        'squad_min_play': [1, 3, 3, 1],
        'squad_max_play': [1, 5 ,5, 3]}
type_data = pd.DataFrame(data, index=[1,2,3,4])

def solve_tff_mp(initial_squad, input_data, md_map, next_md=1, last_md=10, 
                 ta_tot=40, ta_gw=5, objective='regular', decay_base=0.85, transfer_cost=7.5, 
                 exclusions=None, keeps=None, force_transfer_in=None, force_transfer_out=None, no_transfer_mds=None,
                 apply_noise=False, seed_val=None, magnitude=1):
    
    if str(last_md) == last_md:
        if last_md > md_map['unique_dates'].tolist()[-1]:
            last_md = md_map.loc[len(md_map)-1, 'matchday']
        else:
            for i, x in enumerate(md_map['unique_dates'].tolist()):
                if last_md < x:
                    last_md = md_map.loc[i, 'matchday'] - 1
                    break
    if str(last_md) == last_md:
        last_md = md_map.loc[len(md_map)-1, 'matchday']
        
    horizon = last_md + 1 - next_md
    problem_name = f'sky_mp_h{horizon}_d1'
    
    # Sets
    players = input_data.index.tolist()
    element_types = type_data.index.tolist()
    matchdays = list(range(next_md, next_md+horizon))
    all_md = [next_md-1] + matchdays
    
    first_gw = int(md_map.loc[md_map['matchday']==next_md, 'tff_gw'].values[0])
    last_gw = int(md_map.loc[md_map['matchday']==last_md, 'tff_gw'].values[0])
    gameweeks = list(range(first_gw,last_gw+1))
    gw_transfer_allowance = {w: 5 for w in gameweeks}
    gw_transfer_allowance[first_gw] = min(ta_gw,5)

    if apply_noise:
        rng = np.random.default_rng(seed = seed_val)
        input_data['Total_Pts'] = 0
        player_df = pd.read_csv('../data/prior_player_data.csv')
        player_df = player_df[['sky_id', 'noise_factor']]
        input_data = pd.merge(input_data,player_df,on='sky_id').set_index('sky_id')
        for m in matchdays:
            noise = input_data[f'MD_{m}_Pts'] * 0.035 * rng.standard_normal(size = len(input_data)) * magnitude * input_data['noise_factor']
            input_data[f'MD_{m}_Pts'] = input_data[f'MD_{m}_Pts'] + round(noise,2)
            input_data['Total_Pts'] += input_data[f'MD_{m}_Pts']

    # Model
    model = so.Model(name = 'multi_period')

    # Variables
    squad = model.add_variables(players, all_md, name='squad', vartype=so.binary)
    transfer_in = model.add_variables(players, matchdays, name='transfer_in', vartype=so.binary)
    transfer_out = model.add_variables(players, matchdays, name='transfer_out', vartype=so.binary)
    
    # Dictionaries
    squad_type_count = {(t,d): so.expr_sum(squad[p,d] for p in players if input_data.loc[p, 'sky_pos'] == type_data.loc[t, 'sky_pos']) for t in element_types for d in matchdays}
    player_value = (input_data['tff_value']).to_dict()
    bought_amount = {d: so.expr_sum(player_value[p] * transfer_in[p,d] for p in players) for d in matchdays}
    sold_amount = {d: so.expr_sum(player_value[p] * transfer_out[p,d] for p in players) for d in matchdays}
    squad_value = {d: so.expr_sum(player_value[p] * squad[p,d] for p in players) for d in matchdays}
    points_player_day = {(p,d): input_data.loc[p, f'MD_{d}_Pts'] for p in players for d in matchdays}
    squad_count = {d: so.expr_sum(squad[p, d] for p in players) for d in matchdays}
    
    total_number_of_transfers = so.expr_sum(transfer_out[p,d] for p in players for d in matchdays) 

    md_number_of_transfers = {d: so.expr_sum(transfer_out[p,d] for p in players) for d in matchdays}        
    gw_number_of_transfers = {w: so.expr_sum(md_number_of_transfers[d] for d in matchdays if int(md_map.loc[md_map['matchday']==d, 'tff_gw'].values[0]) == w) for w in gameweeks}
    
    # Initial Conditions
    if initial_squad is not None:
        model.add_constraints((squad[p, next_md-1] == 1 for p in initial_squad), name='initial_squad_players')
        model.add_constraints((squad[p, next_md-1] == 0 for p in players if p not in initial_squad), name='initial_squad_others')
    # Constraints: squad
    model.add_constraints((squad_count[d] == 11 for d in matchdays), name='squad_count')    
    # Constraints: formation and budget
    model.add_constraints((squad_type_count[t,d] == [type_data.loc[t, 'squad_min_play'], type_data.loc[t, 'squad_max_play']] for t in element_types for d in matchdays), name='valid_formation_1')
    model.add_constraints((squad_type_count[2,d]-squad_type_count[4,d] <= 3.5 for d in matchdays), name='valid_formation_2')
    model.add_constraints((squad_value[d] <= 50 for d in matchdays), name='squad_budget')
    # Constraints: transfers
    model.add_constraints((squad[p,d] == squad[p,d-1] + transfer_in[p,d] - transfer_out[p,d] for p in players for d in matchdays), name='squad_transfer_rel')
    model.add_constraint(total_number_of_transfers <= min(ta_tot,40), name = 'transfer_allowance')
    model.add_constraints((gw_number_of_transfers[w] <= gw_transfer_allowance[w] for w in gameweeks), name = 'gw_transfer_allowance')
    if no_transfer_mds is not None:
        model.add_constraints((md_number_of_transfers[m] == 0 for m in no_transfer_mds), name='no_transfer_matchdays')
    # Constraints: specified players
    # Force Exclude
    if exclusions is not None:
        model.add_constraints((squad[e, d] == 0 for e in exclusions for d in matchdays), name = 'force_exclude_players')
    # Force Keep
    if keeps is not None:
        model.add_constraints((squad[e, d] == 1 for e in keeps for d in matchdays), name = 'force_keep_players')
    # Force transfer in
    if force_transfer_in is not None:
        model.add_constraints((squad[force_transfer_in[e][0], force_transfer_in[e][1]] == 1 for e in list(range(len(force_transfer_in)))), name = 'force_transfer_in_players')
        model.add_constraints((squad[force_transfer_in[e][0], force_transfer_in[e][1]-1] == 0 for e in list(range(len(force_transfer_in)))), name = 'force_transfer_in_players_2')
    # Force transfer out
    if force_transfer_out is not None:
        model.add_constraints((squad[force_transfer_out[e][0], force_transfer_out[e][1]] == 0 for e in list(range(len(force_transfer_out)))), name = 'force_transfer_out_players')
        model.add_constraints((squad[force_transfer_out[e][0], force_transfer_out[e][1]-1] == 1 for e in list(range(len(force_transfer_out)))), name = 'force_transfer_out_players_2')
    
    # Objective
    md_xp = {d: so.expr_sum(points_player_day[p,d] * (squad[p,d]) for p in players) for d in matchdays}
    if objective == 'regular':
        total_xp = so.expr_sum(md_xp[d] for d in matchdays) - total_number_of_transfers*transfer_cost
        model.set_objective(-total_xp, sense='N', name='total_regular_xp') 
    else:
        # total_xp = so.expr_sum(md_xp[d] * pow(decay_base, d-next_md) for d in matchdays) - total_number_of_transfers*transfer_cost
        days_elapsed0 = md_map.loc[md_map['matchday']==next_md,'days_elapsed'].values[0]
        # Convert weekly decay to daily
        decay_base = decay_base ** (1/7)
        total_xp = so.expr_sum(md_xp[d] * pow(decay_base, md_map.loc[md_map['matchday']==d,'days_elapsed'].values[0]-days_elapsed0) for d in matchdays) - total_number_of_transfers*transfer_cost
        model.set_objective(-total_xp, sense='N', name='total_decay_xp')
    
    # Solve Step
    model.export_mps(filename='skyoutput.mps')
    command = f'cbc skyoutput.mps solve solu {problem_name}_sol.txt'
    # !{command}
    os.system(command)
    # Read the solution back to the file
    with open(f'{problem_name}_sol.txt', 'r') as f:
        for v in model.get_variables():
            v.set_value(0)
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            var = model.get_variable(words[1])
            var.set_value(float(words[2]))
            
    # (OLD) Generate a dataframe to display the solution 
    picks = []
    for d in matchdays:
        for p in players:
            if squad[p,d].get_value() + transfer_out[p,d].get_value() > 0.5:
                lp = input_data.loc[p]
                is_transfer_in = 1 if transfer_in[p,d].get_value() > 0.5 else 0
                is_transfer_out = 1 if transfer_out[p,d].get_value() > 0.5 else 0
                picks.append([
                    int(md_map.loc[md_map['matchday']==d, 'tff_gw'].values[0]), d, lp['fbref_player'], lp['sky_pos'], lp['short_team'], lp['tff_value'], round(points_player_day[p,d], 2), is_transfer_in, is_transfer_out
                ])
    picks_df = pd.DataFrame(picks, columns=['tff_gw','matchday','name', 'pos', 'team', 'value', 'xP', 'transfer_in', 'transfer_out'])#.sort_values(by=['matchday'])
    picks_df.loc[picks_df['matchday'] == next_md, 'transfer_in'] = 0
    
    total_xp = round(so.expr_sum(points_player_day[p,d] * (squad[p,d]) for p in players for d in matchdays).get_value(), 2)
    
    # Generate a better dataframe to display the solution
    plan = []
    for t in element_types:
        for p in players:
            if so.expr_sum(squad[p,d] + transfer_out[p,d] for d in matchdays).get_value() >= 0.5 and input_data.loc[p, 'sky_pos'] == type_data.loc[t, 'sky_pos']:
                lp = input_data.loc[p]
                player_info = [p, lp['short_team'], lp['sky_pos'], lp['tff_value'], lp['fbref_player']]
                for d in matchdays:
                    if squad[p,d].get_value() > 0.5:
                        score = f'{round(points_player_day[p,d], 2)}'
                    else:
                        score = ''
                    player_info.append(score)
                plan.append(player_info)
    columns = ['ID','Team', 'Pos','Value','Name']
    for d in matchdays:
        # w = int(md_map.loc[md_map['matchday']==d, 'tff_gw'].values[0])
        columns.append(f"{d}")
    plan_df = pd.DataFrame(plan, columns=columns)
    plan_df = plan_df.replace(['0.0'],'-')
    plan_df = plan_df.replace(['0.0c'],'-')
    itb_row = ['','','','','ITB']
    for d in matchdays:
        itb = 50 - squad_value[d].get_value()
        itb_row.append(itb)
    plan_df.loc[len(plan_df)] = itb_row

    # make dataframe to record the players in a simulation
    plan = []
    if apply_noise:
        for t in element_types:
            for p in players:
                if so.expr_sum(squad[p,d] + transfer_out[p,d] for d in matchdays).get_value() >= 0.5 and input_data.loc[p, 'sky_pos'] == type_data.loc[t, 'sky_pos']:
                    lp = input_data.loc[p]
                    player_info = [p, lp['short_team'], lp['sky_pos'], lp['tff_value'], lp['fbref_player']]
                    for d in matchdays:
                        if squad[p,d].get_value() > 0.5:
                            score = 1
                        else:
                            score = 0
                        player_info.append(score)
                    plan.append(player_info)
        columns = ['ID','Team', 'Pos','Value','Name']
        for d in matchdays:
            # w = int(md_map.loc[md_map['matchday']==d, 'tff_gw'].values[0])
            columns.append(f"{d}")
        players_in_sim = pd.DataFrame(plan, columns=columns)
    else:
        players_in_sim = None
    
    tff_gw_header = []
    fpl_gw_header = []
    for i in columns:
        if i == 'Name':
            j = 'tff_gw'
            k = 'fpl_gw'
        elif not str(i)[0].isdigit():
            j = ''
            k = ''
        else:
            tff_gw = md_map.loc[md_map['matchday']==int(i),'tff_gw'].values[0]
            fpl_gw = md_map.loc[md_map['matchday']==int(i),'fpl_gw'].values[0]
            j = str(tff_gw)
            k = str(fpl_gw)
        tff_gw_header.append(j)
        fpl_gw_header.append(k)
    plan_df.columns=[tff_gw_header, fpl_gw_header, columns]

    transfers_made = int(total_number_of_transfers.get_value())
    
    return{'model': model, 'picks': picks_df, 'total_xp': total_xp, 'plan': plan_df, 'transfers_made': transfers_made, 'players_in_sim': players_in_sim}

# Produce sensitivity analysis with noise
def solve_tff_mp_noise(initial_squad, input_data, md_map, next_md=1, last_md=7, 
                       ta_tot=40, ta_gw=5, objective='regular', decay_base=0.85, transfer_cost=5, 
                       exclusions=None, keeps=None, force_transfer_in=None, force_transfer_out=None, no_transfer_mds=None,
                       seed_val=None, nsims=5, magnitude=1):
    transfer_sum = 0

    baseline_projection_df = input_data

    for i in range(nsims):
        print(f"Running sim {i+1} of {nsims}...")
        input_data = baseline_projection_df.copy()
        results = solve_tff_mp(initial_squad=initial_squad, input_data=input_data, md_map=md_map, next_md=next_md, last_md=last_md, 
                    ta_tot=ta_tot, ta_gw=ta_gw, objective=objective, decay_base=decay_base, transfer_cost=transfer_cost,
                    exclusions=exclusions, keeps=keeps, force_transfer_in=force_transfer_in, force_transfer_out=force_transfer_out, no_transfer_mds=no_transfer_mds,
                    apply_noise=True, seed_val=seed_val, magnitude=magnitude)
        players_in_sim = results['players_in_sim']
        if i == 0:
            sensitivity_df = players_in_sim
        else:
            rows_to_add = []
            for index, row in results['players_in_sim'].iterrows():
                if row['ID'] in sensitivity_df['ID'].tolist():
                    sensitivity_df.loc[sensitivity_df['ID']==row['ID'], '1':] = sensitivity_df.loc[sensitivity_df['ID']==row['ID'], '1':] + row['1':]
                    continue
                else: 
                    rows_to_add.append(row)
            if len(rows_to_add) > 0:
                rows_to_add_df = pd.concat(rows_to_add, axis=1).T
                sensitivity_df = pd.concat([sensitivity_df, rows_to_add_df], ignore_index=True)
        clear_output(wait=True)
        transfer_sum += results['transfers_made']
    avg_trf = transfer_sum/nsims
    sensitivity_df.loc[:, '1':] = sensitivity_df.loc[:, '1':] * 100 / nsims
    sensitivity_df.loc[:, '1':] = sensitivity_df.loc[:, '1':].astype(int)

    # Sort the dataframe by initial team and position
    sensitivity_df['max_ocuurences'] = 0
    sensitivity_df['init_team'] = 0
    sensitivity_df['pos_code'] = 0
    for index, row in sensitivity_df.iterrows():
        if row['Pos'] == 'GK':
            sensitivity_df.loc[index,'pos_code'] = 1
        elif row['Pos'] == 'DEF':
            sensitivity_df.loc[index,'pos_code'] = 2
        elif row['Pos'] == 'MID':
            sensitivity_df.loc[index,'pos_code'] = 3
        else:
            sensitivity_df.loc[index,'pos_code'] = 4
        if row['ID'] in initial_squad:
            sensitivity_df.loc[index,'init_team'] = 1
    sensitivity_df = sensitivity_df.sort_values(by=['pos_code', 'init_team'], ascending=[True, False])
    sensitivity_df.drop(['max_ocuurences', 'init_team', 'pos_code'], axis=1, inplace=True)
    
    columns = sensitivity_df.columns.values.tolist()

    tff_gw_header = []
    fpl_gw_header = []
    for i in columns:
        if i == 'Name':
            j = 'tff_gw'
            k = 'fpl_gw'
        elif not str(i)[0].isdigit():
            j = ''
            k = ''
        else:
            tff_gw = md_map.loc[md_map['matchday'] == int(i), 'tff_gw'].values[0]
            fpl_gw = md_map.loc[md_map['matchday'] == int(i), 'fpl_gw'].values[0]
            j = str(tff_gw)
            k = str(fpl_gw)
        tff_gw_header.append(j)
        fpl_gw_header.append(k)
    sensitivity_df.columns=[tff_gw_header, fpl_gw_header, columns]

    sens = sensitivity_df.copy()
    sens = sens.style.background_gradient(cmap="RdPu", subset=sensitivity_df.columns[5:]).format(precision=1)
    # sens.set_properties(**{'text-align': 'left'})
    sens = sens.set_table_styles([
                        {'selector': 'th.col_heading', 'props': 'text-align: left;'},
                        {'selector': 'th.col_heading.level0', 'props': 'font-size: 1em;'},
                        {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
                    ], overwrite=False)
                
    return {'sensitivity_df': sens, 'avg_trf': avg_trf, 'sensitivity_df_unformatted': sensitivity_df}

## My Commands
Some example commands are given in the cells below

In [5]:
# To just see the upcoming fixtures:
r0 = generate_ticker()
r0['formatted_fixtures']

In [6]:
# To see a breakdown of EV for a given player and fixture:
prior_player_data = pd.read_csv('../data/prior_player_data.csv')
team_data = pd.read_csv('../data/team_priors.csv')
# Find a players sky_id in prior_player_data.csv
# opp_team is case sensitive, uppercase and lowercase implying home and away respectively
# The below example is Mbeumo at home to Luton
r1 = tff_xP_calc(sky_id=895, opp_team='LUT', prior_player_data=prior_player_data, team_data=team_data, xMins=81, xP_breakdown=True)
r1['xP_breakdown']

,Actions,xP
0,Start,1.80
1,Sub,0.09
2,Goal,1.87
3,Assist,1.12
4,Own Goal,-0.00
5,Pen Goal,0.49
6,Pen Miss,-0.08
8,Yellow Card,-0.11
9,Red Card,-0.01
14,TOTAL,5.17


In [7]:
# To generate player EV from the fixtures:
# Change player or team underlyings as you see fit in prior_player_data.csv and team_priors.csv
# Penalty takers can be overridden in pen_taker_override.csv
# Adding an fplreview.csv to the data folder will let the model use its xMins. Alternatively, edit bl_xmins in prior_player_data.csv to manually override them 

# Example custom fixtures dataframe to add, move, or remove fixtures and assign probabilities of occuring
# Leave 'custom_fixtures=None' to keep the fixtures as they are on the PL site
df = pd.DataFrame(columns=('home_team', 'away_team', 'dates', 'probabilities'))
df.loc[len(df)] = ['FOR', 'SHU', ['2023-12-06'], [1]]

# Generate fixture ticker and player EV
# Setting "teamsheet_boost" to a positive decimal (e.g. 0.05) will boost the EV of players for whom the teamsheet will be released before the deadline
# Set the last matchday, last_md, as an integer, or a date as a string in the format 'YYYY-mm-dd'
r2 = generate_model_output(first_md=1, last_md=42, filename_suffix=None, custom_fixtures=None, teamsheet_boost=0)
display(r2['formatted_fixtures'])
# display(r2['unformatted_fixtures'])
display(r2['skymodel_output'].sort_values(by=['Total_Pts'], ascending=False).head(20))

Using minutes from ../data/fplreview.csv


C:\Users\daadmin\AppData\Local\Temp\ipykernel_27960\4213232137.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fixture_player_data.loc[p, f'MD_{m}_Pts'] = round(xP, 2)
C:\Users\daadmin\AppData\Local\Temp\ipykernel_27960\4213232137.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fixture_player_data.loc[p, f'MD_{m}_Pts'] = round(xP, 2)
C:\Users\daadmin\AppData\Local\Temp\ipykernel_27960\4213232137.py:208: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many time

,sky_id,fbref_player,short_team,sky_pos,tff_value,MD_1_Pts,MD_2_Pts,MD_3_Pts,MD_4_Pts,MD_5_Pts,MD_6_Pts,MD_7_Pts,MD_8_Pts,MD_9_Pts,MD_10_Pts,MD_11_Pts,MD_12_Pts,MD_13_Pts,MD_14_Pts,MD_15_Pts,MD_16_Pts,MD_17_Pts,MD_18_Pts,MD_19_Pts,MD_20_Pts,MD_21_Pts,MD_22_Pts,MD_23_Pts,MD_24_Pts,MD_25_Pts,MD_26_Pts,MD_27_Pts,MD_28_Pts,MD_29_Pts,MD_30_Pts,MD_31_Pts,MD_32_Pts,MD_33_Pts,MD_34_Pts,MD_35_Pts,MD_36_Pts,MD_37_Pts,MD_38_Pts,MD_39_Pts,MD_40_Pts,MD_41_Pts,MD_42_Pts,Total_Pts
429,1280,Erling Haaland,MCI,FOR,8.0,7.33,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,6.26,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7.42,0.00,0.0,0.0,7.64,0.00,0.00,0.00,0.00,6.65,0.00,0.00,0.00,0.0,8.15,0.00,0.00,0.00,0.00,0.00,6.92,0.00,0.00,0.00,50.37
400,351,Mohamed Salah,LIV,FOR,7.2,0.00,0.00,0.00,0.00,0.00,5.40,0.00,0.0,7.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.63,0.0,0.0,0.00,0.00,5.79,0.00,5.81,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,6.34,0.00,0.00,0.00,5.24,0.00,0.00,40.38
56,21,Bukayo Saka,ARS,MID,5.7,0.00,6.13,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,5.01,0.00,0.00,5.75,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,5.39,0.00,0.00,4.93,0.00,0.00,0.0,0.00,0.00,0.00,5.32,0.00,0.00,5.16,0.00,0.00,0.00,37.69
418,417,Marcus Rashford,MUN,FOR,5.9,0.00,0.00,0.00,0.00,0.00,0.00,6.70,0.0,0.00,4.79,0.00,0.00,0.00,0.00,0.00,0.00,6.48,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,4.01,0.00,5.27,0.00,0.00,0.00,0.0,0.00,0.00,4.80,0.00,0.00,0.00,5.31,0.00,0.00,0.00,37.36
212,412,Bruno Fernandes,MUN,MID,6.3,0.00,0.00,0.00,0.00,0.00,0.00,5.39,0.0,0.00,4.37,0.00,0.00,0.00,0.00,0.00,0.00,5.39,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,4.02,0.00,4.73,0.00,0.00,0.00,0.0,0.00,0.00,4.32,0.00,0.00,0.00,4.84,0.00,0.00,0.00,33.06
112,70,Solly March,BHA,MID,3.5,0.00,0.00,5.56,0.00,0.00,0.00,0.00,0.0,4.69,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.97,0.00,0.00,0.0,0.0,0.00,4.18,0.00,0.00,0.00,4.20,0.00,0.00,0.00,0.0,0.00,0.00,0.00,5.04,0.00,4.05,0.00,0.00,0.00,0.00,32.69
422,645,Ollie Watkins,AVL,FOR,5.5,0.00,0.00,0.00,3.95,0.00,0.00,0.00,0.0,0.00,0.00,0.00,6.07,0.00,0.00,0.00,0.00,0.00,0.00,4.64,0.00,0.0,0.0,0.00,0.00,3.91,0.00,0.00,5.02,0.00,0.00,0.00,0.0,4.05,0.00,0.00,0.00,0.00,4.97,0.00,0.00,0.00,0.00,32.61
255,1260,Kaoru Mitoma,BHA,MID,3.6,0.00,0.00,5.53,0.00,0.00,0.00,0.00,0.0,4.60,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.91,0.00,0.00,0.0,0.0,0.00,4.21,0.00,0.00,0.00,4.15,0.00,0.00,0.00,0.0,0.00,0.00,0.00,4.98,0.00,3.92,0.00,0.00,0.00,0.00,32.30
343,895,Bryan Mbeumo,BRE,FOR,3.8,0.00,0.00,0.00,0.00,4.61,0.00,0.00,0.0,4.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.24,0.00,0.00,0.00,0.0,0.0,4.85,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.30,0.0,0.00,4.68,0.00,0.00,0.00,0.00,0.00,0.00,4.15,0.00,29.98
337,810,Martin Ødegaard,ARS,MID,5.1,0.00,4.79,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,3.90,0.00,0.00,4.53,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,4.21,0.00,0.00,4.04,0.00,0.00,0.0,0.00,0.00,0.00,4.26,0.00,0.00,4.12,0.00,0.00,0.00,29.85


In [8]:
# To generate optimal team and plan:
skymodel_output = pd.read_csv('../data/skymodel_output.csv').set_index('sky_id').fillna(0)
# To use FPL Kid data, download the 'Telegraph Fantasy EV - CSV' google sheet as fplkid.csv, add to data folder, and uncomment the following line
#skymodel_output = read_fpl_kid_model(filepath='../data/fplkid.csv')
# Cut off players with low EV to save solve time
ev_cutoff = skymodel_output['Total_Pts'].max() * 0.2
skymodel_output = skymodel_output[skymodel_output['Total_Pts'] > ev_cutoff]

md_map = pd.read_csv('../data/md_map.csv')

# List all the sky IDs of all players to include in the initial squad (or leave empty), see prior_player_data.csv for reference
team = []
exclusions = []

# Change how much you penalize the solver for making a transfer, alternatively set a hard limit by changing the total transfer allowance: ta_tot
transfer_cost =10

# Generate optimal plan
r3 = solve_tff_mp(initial_squad=team, input_data=skymodel_output, md_map=md_map, next_md=1, last_md=42,
                  ta_tot=40, ta_gw=5, objective='regular', decay_base=1, transfer_cost=transfer_cost,
                  exclusions=exclusions, keeps=None, force_transfer_in=None, force_transfer_out=None, no_transfer_mds=None,
                  apply_noise=False)
display(r3['plan'])
print(f"Total xP: {r3['total_xp']}")
print(f"Total transfers made: {r3['transfers_made']}, transfer cost: {transfer_cost}")

NOTE: Initialized model multi_period.


tff_gw     1                          \
                                      fpl_gw     1                           
      ID Team  Pos Value                Name     1     2     3     4     5   
0    373  MCI   GK   4.6             Ederson  4.25     -     -     -     -   
1   1406  BHA  DEF   3.4    Pervis Estupiñán     -     -  4.91     -     -   
2    469  NEW  DEF   3.5        Fabian Schär     -     -     -  4.66     -   
3     20  ARS  DEF   3.9  Gabriel Dos Santos     -  4.89     -     -     -   
4   1411  MUN  MID   3.8            Casemiro     -     -     -     -     -   
5     21  ARS  MID   5.7         Bukayo Saka     -  6.13     -     -     -   
6     70  BHA  MID   3.5         Solly March     -     -  5.56     -     -   
7   1260  BHA  MID   3.6        Kaoru Mitoma     -     -  5.53     -     -   
8    895  BRE  FOR   3.8        Bryan Mbeumo     -     -     -     -  4.61   
9    417  MUN  FOR   5.9     Marcus Rashford     -     -     -     -     -   
10  1280  MCI  FOR   8.0      Erling Haaland  7.33     -     -     -     -   
11                                       ITB   0.3   0.3   0.3   0.3   0.3   

                 2                                      3                   \
                 2                                      3                    
      6     7    8     9    10    11   12   13    14   15   16    17    18   
0     -     -    -     -     -  3.51    -    -     -    -    -     -     -   
1     -     -    -  4.02     -     -    -    -     -    -    -     -  3.88   
2     -     -    -     -     -   1.8    -    -     -    -    -     -     -   
3     -     -    -     -     -     -    -    -  3.74    -    -  4.34     -   
4     -  4.59    -     -  4.09     -    -    -     -    -    -   4.3     -   
5     -     -    -     -     -     -    -    -  5.01    -    -  5.75     -   
6     -     -    -  4.69     -     -    -    -     -    -    -     -  4.97   
7     -     -    -   4.6     -     -    -    -     -    -    -     -  4.91   
8     -     -    -  4.15     -     -    -    -     -    -    -  4.24     -   
9     -   6.7    -     -  4.79     -    -    -     -    -    -  6.48     -   
10    -     -    -     -     -  6.26    -    -     -    -    -     -     -   
11  0.3   0.3  0.3   0.3   0.3   0.3  0.3  0.3   0.3  0.3  0.3   0.3   0.3   

                  4                                5                         \
                  4                                5                          
      19    20   21   22    23    24   25    26   27    28    29   30    31   
0    4.6     -    -    -  4.25     -    -     -    -  3.83     -    -     -   
1      -     -    -    -     -  2.97    -     -    -  2.48     -    -     -   
2      -  3.72    -    -     -   3.0    -     -    -     -     -    -  4.72   
3      -     -    -    -     -     -    -  3.47    -     -  3.85    -     -   
4      -     -    -    -     -     -    -  3.86    -  4.22     -    -     -   
5      -     -    -    -     -     -    -  5.39    -     -  4.93    -     -   
6      -     -    -    -     -  4.18    -     -    -   4.2     -    -     -   
7      -     -    -    -     -  4.21    -     -    -  4.15     -    -     -   
8      -     -    -    -  4.85     -    -     -    -     -     -    -   3.3   
9      -     -    -    -     -     -    -  4.01    -  5.27     -    -     -   
10  7.42     -    -    -  7.64     -    -     -    -  6.65     -    -     -   
11   0.3   0.3  0.3  0.3   0.3   0.3  0.3   0.3  0.3   0.3   0.3  0.3   0.3   

            6                            7                        
            6                            7                        
     32    33    34    35    36   37    38    39   40    41   42  
0     -  4.65     -     -     -    -     -  4.44    -     -    -  
1     -     -     -     -  4.31    -  3.04     -    -     -    -  
2     -     -     -     -     -  4.7     -   5.3    -     -    -  
3     -     -     -     -  3.58    -     -  3.91    -     -    -  
4     -     -     -  3.86     -    -     

Total xP: 360.54
Total transfers made: 0, transfer cost: 10


In [9]:
# To run multiple solves with noise:
skymodel_output = pd.read_csv('../data/skymodel_output.csv').set_index('sky_id').fillna(0)
# To use FPL Kid data, download the 'Telegraph Fantasy EV - CSV' google sheet as fplkid.csv, add to data folder, and uncomment the following line
#skymodel_output = read_fpl_kid_model(filepath='../data/fplkid.csv')
ev_cutoff = skymodel_output['Total_Pts'].max() * 0.2
skymodel_output = skymodel_output[skymodel_output['Total_Pts'] > ev_cutoff]
md_map = pd.read_csv('../data/md_map.csv')

team = [1280]
transfer_cost = 5
# Choose a number of simulations to run
nsims = 5
# Choose the relative magnitude of applied noise (1 is standard)
magnitude = 1

# Generate sensitivity analysis
r4 = solve_tff_mp_noise(initial_squad=team, input_data=skymodel_output, md_map=md_map, next_md=1, last_md=42,
                        ta_tot=40, ta_gw=5, objective='regular', decay_base=1, transfer_cost=transfer_cost,
                        exclusions=None, keeps=None, force_transfer_in=None, force_transfer_out=None, no_transfer_mds=None,
                        seed_val=None, nsims=nsims, magnitude=magnitude)
display(r4['sensitivity_df'])
# display(r4['sensitivity_df_unformatted'])
print(f"Number of sims: {nsims}, average transfers made: {r4['avg_trf']}, with cost: {transfer_cost}")


Number of sims: 5, average transfers made: 16.4, with cost: 5
